In [124]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import misc as ms
from fitter import model_average as md
from fitter import fitter as fit

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [131]:
reload(fit)
reload(dl)
reload(ms)
reload(md)

<module 'fitter.model_average' from '../fitter\model_average.pyc'>

In [132]:
data_loader = dl.data_loader()
fit_results = data_loader.get_fit_info('fit_results.csv')
other_results = data_loader.get_fit_info('other_results.csv')

In [133]:
model_average = md.model_average(fit_results)

In [134]:
model_average.average(split_unc=True)

[1.1810713572265046, 0.002904502136415246, 0.0036373799321951132]

In [135]:
model_average.average('delta_su2')

-0.00383(69)

### Make plots

In [71]:
model_average.plot_fits('a');

In [72]:
model_average.plot_histogram('delta_su2');

In [73]:
model_average.plot_comparison(other_results=other_results, param='FK/Fpi_pm');